In [1]:
import shap
import pandas as pd
import pickle
import os

In [2]:
os.chdir('../')

In [3]:
X = pd.read_csv('data/processed/dataset.csv')

In [4]:
with open('models/rf_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

In [5]:
X = X[rf_model.feature_names_in_]

In [6]:
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X)

In [7]:
print(f"length of shap_values: {len(shap_values)} \n")
print(f"shape of elements within shap_values: {[i.shape for i in shap_values]} \n")

length of shap_values: 116 

shape of elements within shap_values: [(10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 2), (10, 

In [8]:
shap.summary_plot(shap_values[1], X, plot_type='dot', show=False, plot_size=[10,6])

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

In [ ]:
shap.dependence_plot(X.columns.get_indexer(['FEVI post']), shap_values[1],
                     X, interaction_index=False)

In [ ]:
shap.dependence_plot(X.columns.get_indexer(['SvO2']), shap_values[1],
                     X, interaction_index=False)

In [ ]:
shap.dependence_plot(X.columns.get_indexer(['Lactato post']), shap_values[1],
                     X, interaction_index=False)